In [ ]:
!git clone -b train --single-branch https://github.com/MLOpshub/extractive-qa-mlops.git
%cd extractive-qa-mlops

!pip -q install uv
!uv sync


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


In [ ]:
import os

os.environ["MODELS_DIR"] = "/content/drive/MyDrive/extractive-qa-mlops/artifacts/models"
os.environ["MLRUNS_DIR"] = "/content/drive/MyDrive/extractive-qa-mlops/mlruns"


In [ ]:
!uv run python -m src.train --config configs/train.yaml


MLflow UI

```powershell
uv run python -m mlflow ui --backend-store-uri "file:./mlruns" --port 5000
```